# Example of a complex environment for an MPI-aware task

In [ ]:
import sys
from jobqueue_features.clusters import CustomSLURMCluster
from jobqueue_features.decorators import on_cluster, mpi_task
from jobqueue_features.mpi_wrapper import SRUN

## Simplify getting environment
Let's use a feature of Lmod that allows us to restore a saved environment with
```bash
module restore <module_set>
```
The big plus of this is that we can maintain this environment separately to our python code.

In [ ]:
lammps_cluster = CustomSLURMCluster(
    name="LAMMPS_cluster",
    walltime="00:05:00",
    queue = "devel",
    nodes=2,
    ntasks_per_node=12,
    mpi_mode=True,
    mpi_launcher=SRUN,
    maximum_scale=2,
    env_extra=[
        "module restore jobqueue_tutorial",
    ],
)

# Run a task for some time on the remote resources
Be careful to remember that you only have access to the memory space of the root MPI process

In [ ]:
@mpi_task(cluster_id=lammps_cluster.name)
def task(input_file=None, run_steps=100):
    from mpi4py import MPI
    from lammps import PyLammps


    L = PyLammps()
    if input_file:
        L.file(input_file)
    else:
        L.file("/p/project/cecam/shared/in.melt")

    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    # This only appears in the slurm job output
    return_string = "Hi, my rank is %d" % rank

    # Simulate the system for a few steps
    L.run(run_steps)
    
    if rank==0:
        return_string = "Potential energy: %s" % L.eval("pe")

    # The flush is required to ensure that the print statements appear in the job log
    # files
    print(return_string)
    sys.stdout.flush()

    return return_string

In [ ]:
@on_cluster(cluster=lammps_cluster, cluster_id=lammps_cluster.name)
def main(input_file=None, run_steps=100):
    t1 = task(input_file=input_file, run_steps=run_steps)
    print(t1.result())

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.DEBUG)

In [ ]:
main(input_file="/p/project/cecam/shared/in.melt", run_steps=1000)

In [ ]:
!squeue -u $USER -l

Check the joblogs to see that LAMMPS doesn't appear to be a well behaved task in this case (or Dask needs some help to tolerate it's bad behaviour)